In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.random_agent import RandomAgent
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [3]:
#df = pd.read_csv('../../data/anemia_synth_dataset_some_nans_unspecified_more_feats.csv')
#df = pd.read_csv('../../data/more_features/more_feats_new_labels_0.1.csv')
df = pd.read_csv('../../data/more_features//more_feats_correlated_missing_4.csv')
df = df.fillna(-1)
df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.183192,187.573466,4.951674,-1.000000,316.593436,-1.000000,-1.0,3.531295,1,-1.000000,-1.000000,96.721542,49.530542,23.186628,-1.0,-1.000000,57.247868,Hemolytic anemia
1,14.387445,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,1,-1.000000,98.357508,112.758764,62.464566,7.168920,-1.0,-1.000000,-1.000000,No anemia
2,12.749357,-1.000000,-1.000000,6.179371,498.418768,76.759285,-1.0,-1.000000,1,1.022939,-1.000000,75.739552,72.072041,20.600875,-1.0,38.248071,32.068372,Iron deficiency anemia
3,11.508870,197.180945,1.200125,-1.000000,-1.000000,102.900301,-1.0,-1.000000,1,-1.000000,-1.000000,66.999185,18.353272,-1.000000,-1.0,34.526609,28.702050,Unspecified anemia
4,9.456656,427.952052,-1.000000,0.660252,-1.000000,104.543774,-1.0,2.713693,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,28.369968,-1.000000,Vitamin B12/Folate deficiency anemia


In [4]:
class_dict = constants.CLASS_DICT
df['label'] = df['label'].replace(class_dict)
X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

#### The Random Agent

In [5]:
%%time
rand_agent = RandomAgent(X_test[0:], y_test[0:])
test_df = rand_agent.test()
test_df.head()

Testing done.....
Wall time: 3min 22s


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,2.0,0.0,0.0,-1.0,0.0,"[ret_count, Vitamin B12/Folate deficiency anemia]",6.0,1.0
1,1.0,1.0,0.0,-1.0,0.0,[No anemia],2.0,0.0
2,2.0,2.0,0.0,-1.0,0.0,"[segmented_neutrophils, Iron deficiency anemia]",2.0,4.0
3,1.0,3.0,0.0,-1.0,0.0,[Anemia of chronic disease],5.0,3.0
4,1.0,4.0,0.0,-1.0,0.0,[Anemia of chronic disease],5.0,3.0


In [6]:
# %%time
# rand_agent.test_sample(2)

In [7]:
# from datetime import datetime
# start = datetime.now()
# rand_agent.test_sample(2)
# end = datetime.now()
# duration = end-start
# duration

In [8]:
len(X_test), len(test_df)

(14000, 14000)

In [9]:
# test_df.to_csv('../../test_dfs/many_features/random_test_df_noisy_0.4.csv', index=False)

In [10]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

11.807142857142857

In [11]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(1.5399285714285715, -0.7715714285714286)

In [12]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(0.11807142857142858, 0.11805631932664483, 0.4980336985210257)

In [13]:
test_df.y_pred.unique()

array([1., 0., 4., 3., 2., 6., 7., 5.])